In [621]:
#Suzan Iloglu, May 21,2020
import gurobipy as gp
import csv
from gurobipy import GRB
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200

In [622]:
from IPython.display import Image

## Social Vulnerability Index for Mapping Community Health Worker Need
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a>

### I. Importing SVI data and computing state percentile ranking
The CDC uses both a USA-wide percentile ranking and a state by state percentile ranking. For our project given that funding is likely going to be managed at a state level using a state percentile ranking makes the most sense and will be most sensitive to regional socioeconomic differences. 

In [623]:
## import SVI data
svi_census = gpd.read_file("Data/SVI2018_US_tract.geojson")
#svi_tribal = gpd.read_file("Data/SVI2018_tribal_tract.geojson")
svi_puerto_rico_census = gpd.read_file("Data/SVI2018_PuertoRico_tract.geojson")


In [624]:
## import svi data by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)
svi_counties.head(5)
svi_counties.shape

(3220, 126)

In [625]:
svi_counties_puerto_rico.head(1)
svi_puerto_rico_census.rename(columns={'SPL_THEME1': 'SPL_THEMES'})
svi_counties_puerto_rico.rename(columns={'SPL_THEME1': 'SPL_THEMES'})

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEMES,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry
0,72,PUERTO RICO,PR,Aibonito,72009,"Aibonito Municipio, Puerto Rico",31.310735,23457,0,9800,196,8301,265,10768,894,376,151,10630,851,3884,369,4439,0,4693,0,6117,499,1204,221.6,23415,46.0,16811,836.2,117,45.2,35,34,331,110.6,1202,215,27,31,45.9,3.8,5.4,2.1,10630.0,851.0,23.4,2.2,18.9,0.0,20.0,0.0,26.1,2.1,14.5,2.6,99.8,0.2,75.3,3.7,1.2,0.5,0.4,0.3,4.0,1.3,14.5,2.5,0.1,0.1,0.3636,0.0260,0.4416,0.2208,1.0520,0.1818,0.5714,0.3117,0.7922,0.6364,2.3117,0.7403,0.7273,0.8442,1.5715,0.8831,0.2338,0.6753,0.7143,0.4026,0.0649,2.0909,0.3377,7.0261,0.3766,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,682,263,2.9,1.1,-999,0.006922,0.402665,"POLYGON ((-66.31804 18.13059, -66.31573 18.134..."
1,72,PUERTO RICO,PR,Añasco,72011,"Añasco Municipio, Puerto Rico",39.284885,27368,0,12524,193,8933,359,11831,939,1290,280,10259,693,5955,423,5234,0,5307,0,6914,567,965,202.6,26494,273.0,17923,867.1,224,97.2,18,28,117,76.8,1154,230,122,104,43.4,3.4,12.9,2.8,10259.0,693.0,30.7,2.2,19.1,0.0,19.4,0.0,25.3,2.1,10.8,2.2,96.8,1.0,68.3,3.3,1.8,0.8,0.1,0.2,1.3,0.9,12.9,2.5,0.4,0.4,0.2468,0.2597,0.4805,0.7532,1.7402,0.4026,0.6104,0.1169,0.7403,0.1948,1.6624,0.2597,0.0519,0.5065,0.5584,0.2597,0.3896,0.2208,0.0130,0.1688,0.4416,1.2338,0.0519,5.1948,0.0779,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1023,272,3.7,1.0,-999,0.008779,0.536131,"POLYGON ((-67.22532 18.29689, -67.22265 18.303..."
2,72,PUERTO RICO,PR,Arecibo,72013,"Arecibo Municipio, Puerto Rico",125.938590,87242,0,41309,402,31788,653,41504,1773,4910,558,10203,499,14979,749,17765,0,17035,0,17640,865,4062,459.9,86776,191.0,52381,1548.4,1820,274.0,22,29,644,168.2,5395,415,1217,217,48.0,2.1,17.6,1.9,10203.0,499.0,24.3,1.2,20.4,0.0,19.5,0.0,20.4,1.0,12.8,1.4,99.5,0.2,62.9,1.9,4.4,0.7,0.1,0.1,2.0,0.5,17.0,1.3,1.4,0.2,0.4805,0.4935,0.5065,0.3117,1.7922,0.4416,0.7792,0.1429,0.4026,0.3896,1.7143,0.3117,0.5065,0.2597,0.7662,0.3377,0.7662,0.2208,0.1688,0.6883,0.8701,2.7142,0.6883,6.9869,0.3636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4454,612,5.1,0.7,-999,0.028150,0.790736,"POLYGON ((-66.77066 18.43595, -66.76832 18.443..."
3,72,PUERTO RICO,PR,Barceloneta,72017,"Barceloneta Municipio, Puerto Rico",18.695792,24299,0,10094,209,8010,318,12184,1135,1339,381,9170,808,4442,643,4426,0,5183,0,4974,734,1299,287.2,24262,40.0,15184,1250.9,50,54.1,20,32,200,124.7,1100,254,18,21,50.2,4.7,18.4,4.7,9170.0,808.0,26.6,3.8,18.2,0.0,21.3,0.0,20.5,3.0,16.2,3.5,99.8,0.2,65.9,5.4,0.5,0.5,0.2,0.3,2.5,1.6,13.7,3.2,0.1,0.1,0.5455,0.5195,0.6883,0.4416,2.1949,0.6104,0.4416,0.6753,0.4416,0.8442,2.4027,0.8182,0.7273,0.4286,1.1559,0.6364,0.0519,0.3636,0.3636,0.2857,0.0649,1.1297,0.0390,6.8832,0.3506,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1379,441,5.7,1.8,-999,0.004187,0.408268,"POLYGON ((-66.59346 18.39941, -66.59102 18.410..."
4,72,PUERTO RICO,PR,Caguas,72025,"Caguas Municipio, Puerto Rico",58.602617,131363,0,57739,445,48629,835,48880,2329,7475,768,14654,624

In [626]:
## replacing -999 values with nan for calculations
svi_census = pd.concat([svi_census, svi_puerto_rico_census], sort = False)
svi_county = svi_counties.fillna(-999)
svi_census = svi_census.fillna(-999)
#svi_county.head(5)


In [627]:
svi_puerto_rico_census.head(10)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry
0,72,PUERTO RICO,PR,72023,Cabo Rojo,72023980000,"Census Tract 9800, Cabo Rojo Municipio, Puerto...",1.896548,0,14,0,14,0,14,0,14,0,14,-999,-999,0,14,0,14,0,14,0,14,0,19.8,0,19.8,0,56.0,0,19.8,0,14,0,19.8,0,14,0,14,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0000,-999.000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,14,-999.0,-999.0,-999,0.000711,0.266443,"MULTIPOLYGON (((-67.20000 17.95858, -67.19675 ..."
1,72,PUERTO RICO,PR,72031,Carolina,72031980003,"Census Tract 9800.03, Carolina Municipio, Puer...",2.245703,0,14,0,14,0,14,0,14,0,14,-999,-999,0,14,0,14,0,14,0,14,0,19.8,0,19.8,0,56.0,0,19.8,0,14,0,19.8,0,14,0,14,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0000,-999.000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,14,-999.0,-999.0,-999,0.000587,0.117558,"POLYGON ((-66.02121 18.43873, -66.01428 18.441..."
2,72,PUERTO RICO,PR,72033,Cataño,72033980001,"Census Tract 9800.01, Cata�o Municipio, Puerto...",0.216514,0,14,0,14,0,14,0,14,0,14,-999,-999,0,14,0,14,0,14,0,14,0,19.8,0,19.8,0,56.0,0,19.8,0,14,0,19.8,0,14,0,14,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0000,-999.000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,14,-999.0,-999.0,-999,0.000048,0.030038,"POLYGON ((-66.14688 18.45486, -66.14458 18.458..."
3,72,PUERTO RICO,PR,72033,Cataño,72033980002,"Census Tract 9800.02, Cata�o Municipio, Puerto...",0.418173,0,14,0,14,0,14,0,14,0,14,-999,-999,0,14,0,14,0,14,0,14,0,19.8,0,19.8,0,56.0,0,19.8,0,14,0,19.8,0,14,0,14,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0000,-999.000,-999.0,-999

In [628]:
svi_census.head(1)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry
0,01,ALABAMA,AL,01015,Calhoun,01015981901,"Census Tract 9819.01, Calhoun County, Alabama",24.243865,0,12,0,12,0,12,0,12,0,12,-999,-999,0,12,0,12,0,12,0,12,0,17.0,0,17.0,0,48.0,0,17.0,0,12,0,17.0,0,12,0,12,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,12,-999.0,-999.0,656,0.006105,0.3394,"POLYGON ((-86.01323 33.68426, -86.01224 33.687..."


In [629]:

svi_county[svi_county['STATE'] == "NEW YORK"]


,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry
741,36,NEW YORK,NY,Chenango,36017,"Chenango County, New York",893.592113,48348,0,25448,84,20616,341,6796,635,1327,203,26717,717,4076,321,9540,53,10141,56,9174,507,1850,231.4,2502,11.0,237,102.1,694,140.7,5293,312,488,134.5,1650,227,686,138,14.3,1.3,5.7,0.9,26717.0,717.0,11.8,0.9,19.7,0.1,21.0,0.1,19.2,1.1,9.0,1.1,5.2,0.0,0.5,0.2,2.7,0.6,20.8,1.2,2.4,0.7,8.0,1.1,1.4,0.3,0.4653,0.5344,0.4656,0.4756,1.9409,0.4901,0.6578,0.3216,0.7864,0.6399,2.4057,0.7985,0.1366,0.3582,0.4948,0.2305,0.4677,0.7997,0.6313,0.7867,0.3018,2.9872,0.7571,7.8286,0.5646,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2261,391,4.7,0.8,44025,0.254881,2.359425,"POLYGON ((-75.88983 42.72384, -75.82109 42.727..."
742,36,NEW YORK,NY,Columbia,36021,"Columbia County, New York",634.663171,60919,0,33386,142,25243,448,7039,697,1537,239,35581,1190,4392,416,13577,40,10965,0,9367,601,1944,285.7,8148,145.0,804,203.4,1226,203.8,2630,272,349,121.2,1808,233,2487,305,11.9,1.2,4.9,0.7,35581.0,1190.0,9.6,0.9,22.3,0.1,18.0,0.0,15.8,1.0,7.7,1.1,13.4,0.2,1.4,0.3,3.7,0.6,7.9,0.8,1.4,0.5,7.2,0.9,4.1,0.5,0.3061,0.3981,0.0815,0.3114,1.0971,0.2322,0.8332,0.0847,0.5365,0.4126,1.8670,0.3830,0.4333,0.6718,1.1051,0.5727,0.5931,0.3760,0.2964,0.7112,0.7635,2.7402,0.6320,6.8094,0.3854,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2759,387,4.7,0.7,50727,0.183247,1.917350,"POLYGON ((-73.92911 42.08166, -73.92668 42.090..."
743,36,NEW YORK,NY,Cortland,36023,"Cortland County, New York",498.772655,48123,0,20766,140,17685,313,6884,595,1113,212,26979,1234,2800,334,7547,67,9436,159,6249,434,1576,217.1,3544,108.0,165,109.2,1158,202.8,1518,148,242,71.9,1475,212,3546,326,15.5,1.3,4.5,0.8,26979.0,1234.0,9.3,1.1,15.7,0.1,19.6,0.3,13.1,0.9,8.9,1.2,7.4,0.2,0.4,0.2,5.6,1.0,7.3,0.7,1.4,0.4,8.3,1.2,7.4,0.7,0.5471,0.3299,0.4459,0.2913,1.6142,0.3908,0.2665,0.1853,0.2649,0.6234,1.3401,0.1076,0.2518,0.2970,0.5488,0.2601,0.7348,0.3477,0.2964,0.8147,0.8848,3.0784,0.7969,6.5815,0.3452,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2056,409,4.3,0.9,46157,0.142472,1.514133,"POLYGON ((-76.27467 42.77126, -76.19594 42.774..."
744,36,NEW YORK,NY,Delaware,36025,"Delaware County, New York",1442.419683,45502,0,31565,137,19030,493,7000,799,1097,208,26629,888,3875,363,10595,53,7854,68,7324,505,1564,231.2,3544,43.0,462,181.0,707,140.4,4202,336,513,137.9,1575,191,2330,264,16.2,1.9,5.1,1.0,26629.0,888.0,11.8,1.1,23.3,0.1,17.3,0.1,16.3,1.1,8.2,1.2,7.8,0.1,1.1,0.4,2.2,0.4,13.3,1.1,2.7,0.7,8.3,1.0,5.1,0.6,0.5962,0.4303,0.4710,0.4756,1.9731,0.5016,0.8711,0.0605,0.5756,0.5053,2.0125,0.4954,0.2668,0.6087,0.8755,0.4374,0.4037,0.5957,0.7065,0.8147,0.8131,3.3337,0.8991,8.1948,0.6331,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2721,326,6.0,0.7,41735,0.414296,3.021171,"POLYGON ((-75.42264 42.05591, -75.42262 42.055..."
745,36,NEW YORK,NY,Dutchess,36027,"Dutchess County, New York",795.647042,293894,0,120472,405,107347,824,25335,1615,8981,735,38048,621,19326,1004,487

In [630]:
## calculating state by state percentile rank for tract svi
svi_census['EPL_POV_ST'] = svi_census.groupby('STATE')['EP_POV'].rank(pct=True)
svi_census['EPL_UNEMP_ST'] = svi_census.groupby('STATE')['EP_UNEMP'].rank(pct=True)
svi_census['EPL_PCI_ST'] = svi_census.groupby('STATE')['EP_PCI'].rank(pct=True)
svi_census['EPL_NOHSDP_ST'] = svi_census.groupby('STATE')['EP_NOHSDP'].rank(pct=True)

svi_census['SPL_THEME1_ST'] = svi_census['EPL_POV_ST'] + svi_census['EPL_UNEMP_ST'] + svi_census['EPL_PCI_ST'] + svi_census['EPL_NOHSDP_ST']

svi_census['RPL_THEME1_ST'] = svi_census.groupby('STATE')['SPL_THEME1_ST'].rank(pct=True)
svi_census['EPL_AGE65_ST'] = svi_census.groupby('STATE')['EP_AGE65'].rank(pct=True)
svi_census['EPL_AGE17_ST'] = svi_census.groupby('STATE')['EP_AGE17'].rank(pct=True)
svi_census['EPL_DISABL_ST'] = svi_census.groupby('STATE')['EP_DISABL'].rank(pct=True)
svi_census['EPL_SNGPNT_ST'] = svi_census.groupby('STATE')['EP_SNGPNT'].rank(pct=True)

svi_census['SPL_THEME2_ST'] = svi_census['EPL_AGE65_ST'] + svi_census['EPL_AGE17_ST'] + svi_census['EPL_DISABL_ST'] + svi_census['EPL_SNGPNT_ST']

svi_census['RPL_THEME2_ST'] = svi_census.groupby('STATE')['SPL_THEME2_ST'].rank(pct=True)
svi_census['EPL_MINRTY_ST'] = svi_census.groupby('STATE')['EP_MINRTY'].rank(pct=True)
svi_census['EPL_LIMENG_ST'] = svi_census.groupby('STATE')['EP_LIMENG'].rank(pct=True)

svi_census['SPL_THEME3_ST'] = svi_census['EPL_MINRTY_ST'] + svi_census['EPL_LIMENG_ST']

svi_census['RPL_THEME3_ST'] = svi_census.groupby('STATE')['SPL_THEME3_ST'].rank(pct=True)
svi_census['EPL_MUNIT_ST'] = svi_census.groupby('STATE')['EP_MUNIT'].rank(pct=True)
svi_census['EPL_MOBILE_ST'] = svi_census.groupby('STATE')['EP_MOBILE'].rank(pct=True)
svi_census['EPL_CROWD_ST'] = svi_census.groupby('STATE')['EP_CROWD'].rank(pct=True)
svi_census['EPL_NOVEH_ST'] = svi_census.groupby('STATE')['EP_NOVEH'].rank(pct=True)
svi_census['EPL_GROUPQ_ST'] = svi_census.groupby('STATE')['EP_GROUPQ'].rank(pct=True)

svi_census['SPL_THEME4_ST'] = svi_census['EPL_MUNIT_ST'] + svi_census['EPL_MOBILE_ST'] + svi_census['EPL_CROWD_ST'] + svi_census['EPL_NOVEH_ST'] + svi_census['EPL_GROUPQ_ST']

svi_census['RPL_THEME4_ST'] = svi_census.groupby('STATE')['SPL_THEME4_ST'].rank(pct=True)

svi_census['SPL_THEMES_ST'] = svi_census['SPL_THEME1_ST'] + svi_census['SPL_THEME2_ST'] + svi_census['SPL_THEME3_ST'] + svi_census['SPL_THEME4_ST']

svi_census['RPL_ThemesStates'] = svi_census.groupby('STATE')['SPL_THEMES_ST'].rank(pct=True)


#svi_census['RPL_ThemesStates'] = svi_census.groupby('STATE')['SPL_THEMES'].rank(pct=True)
#svi_census['RPL_Theme1States'] = svi_census.groupby('STATE')['SPL_THEME1'].rank(pct=True)
#svi_census['RPL_Theme2States'] = svi_census.groupby('STATE')['SPL_THEME2'].rank(pct=True)
#svi_census['RPL_Theme3States'] = svi_census.groupby('STATE')['SPL_THEME3'].rank(pct=True)
#svi_census['RPL_Theme4States'] = svi_census.groupby('STATE')['SPL_THEME4'].rank(pct=True)

svi_census.head(5)


,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry,EPL_POV_ST,EPL_UNEMP_ST,EPL_PCI_ST,EPL_NOHSDP_ST,SPL_THEME1_ST,RPL_THEME1_ST,EPL_AGE65_ST,EPL_AGE17_ST,EPL_DISABL_ST,EPL_SNGPNT_ST,SPL_THEME2_ST,RPL_THEME2_ST,EPL_MINRTY_ST,EPL_LIMENG_ST,SPL_THEME3_ST,RPL_THEME3_ST,EPL_MUNIT_ST,EPL_MOBILE_ST,EPL_CROWD_ST,EPL_NOVEH_ST,EPL_GROUPQ_ST,SPL_THEME4_ST,RPL_THEME4_ST,SPL_THEMES_ST,RPL_ThemesStates
0,01,ALABAMA,AL,01015,Calhoun,01015981901,"Census Tract 9819.01, Calhoun County, Alabama",24.243865,0,12,0,12,0,12,0,12,0,12,-999,-999,0,12,0,12,0,12,0,12,0,17.0,0,17.0,0,48.0,0,17.0,0,12,0,17.0,0,12,0,12,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0000,-999.0000,-999.000,-999.0,-999.000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,12,-999.0,-999.0,656,0.006105,0.339400,"POLYGON ((-86.01323 33.68426, -86.01224 33.687...",0.002966,0.002542,0.002542,0.002966,0.011017,0.002542,0.002542,0.002966,0.002542,0.007203,0.015254,0.002542,0.002542,0.205508,0.208051,0.002542,0.171186,0.002966,0.119492,0.002966,0.257627,0.554237,0.002542,0.788559,0.002542
1,01,ALABAMA,AL,01015,Calhoun,01015981902,"Census Tract 9819.02, Calhoun County, Alabama",34.646714,0,12,0,12,0,12,0,12,0,12,-999,-999,0,12,0,12,0,12,0,12,0,17.0,0,17.0,0,48.0,0,17.0,0,12,0,17.0,0,12,0,12,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0000,-999.0000,-999.000,-999.0,-999.000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,12,-999.0,-999.0,146,0.008752,0.420459,"POLYGON ((-86.01730 33.69662, -86.01704 33.698...",0.002966,0.002542,0.002542,0.002966,0.011017,0.002542,0.002542,0.002966,0.002542,0.007203,0.015254,0.002542,0.002542,0.205508,0.208051,0.002542,0.171186,0.002966,0.119492,0.002966,0.257627,0.554237,0.002542,0.788559,0.002542
2,01,ALABAMA,AL,01015,Calhoun,01015981903,"Census Tract 9819.03, Calhoun County, Alabama",13.926276,0,12,0,12,0,12,0,12,0,12,-999,-999,0,12,0,12,0,12,0,12,0,17.0,0,17.0,0,48.0,0,17.0,0,12,0,17.0,0,12,0,12,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0000,-999.0000,-999.

In [631]:
# Create dictionary for census tract SVI
svi_census['FIPS'] = svi_census['FIPS'].astype(int)
svi_census['STCNTY'] = svi_census['STCNTY'].astype(int)
SVI_census = dict(zip(svi_census.FIPS, svi_census.RPL_ThemesStates))

In [632]:
## exporting tracts svi to geojson
# svi_census.to_file("Data/svi_2018_tracts_state_ranked.geojson",driver='GeoJSON')

In [633]:
## calculating state by state percentile rank for county svi

#svi_county['RPL_ThemesStatesCt'] = svi_county.groupby('STATE')['SPL_THEMES'].rank(pct=True)
#svi_county['RPL_Theme1StatesCt'] = svi_county.groupby('STATE')['SPL_THEME1'].rank(pct=True)
#svi_county['RPL_Theme2StatesCt'] = svi_county.groupby('STATE')['SPL_THEME2'].rank(pct=True)
#svi_county['RPL_Theme3StatesCt'] = svi_county.groupby('STATE')['SPL_THEME3'].rank(pct=True)
#svi_county['RPL_Theme4StatesCt'] = svi_county.groupby('STATE')['SPL_THEME4'].rank(pct=True)

svi_county['EPL_POV_ST'] = svi_county.groupby('STATE')['EP_POV'].rank(pct=True)
svi_county['EPL_UNEMP_ST'] = svi_county.groupby('STATE')['EP_UNEMP'].rank(pct=True)
svi_county['EPL_PCI_ST'] = svi_county.groupby('STATE')['EP_PCI'].rank(pct=True)
svi_county['EPL_NOHSDP_ST'] = svi_county.groupby('STATE')['EP_NOHSDP'].rank(pct=True)

svi_county['SPL_THEME1_ST'] = svi_county['EPL_POV_ST'] + svi_county['EPL_UNEMP_ST'] + svi_county['EPL_PCI_ST'] + svi_county['EPL_NOHSDP_ST']

svi_county['RPL_THEME1_ST'] = svi_county.groupby('STATE')['SPL_THEME1_ST'].rank(pct=True)
svi_county['EPL_AGE65_ST'] = svi_county.groupby('STATE')['EP_AGE65'].rank(pct=True)
svi_county['EPL_AGE17_ST'] = svi_county.groupby('STATE')['EP_AGE17'].rank(pct=True)
svi_county['EPL_DISABL_ST'] = svi_county.groupby('STATE')['EP_DISABL'].rank(pct=True)
svi_county['EPL_SNGPNT_ST'] = svi_county.groupby('STATE')['EP_SNGPNT'].rank(pct=True)

svi_county['SPL_THEME2_ST'] = svi_county['EPL_AGE65_ST'] + svi_county['EPL_AGE17_ST'] + svi_county['EPL_DISABL_ST'] + svi_county['EPL_SNGPNT_ST']

svi_county['RPL_THEME2_ST'] = svi_county.groupby('STATE')['SPL_THEME2_ST'].rank(pct=True)
svi_county['EPL_MINRTY_ST'] = svi_county.groupby('STATE')['EP_MINRTY'].rank(pct=True)
svi_county['EPL_LIMENG_ST'] = svi_county.groupby('STATE')['EP_LIMENG'].rank(pct=True)

svi_county['SPL_THEME3_ST'] = svi_county['EPL_MINRTY_ST'] + svi_county['EPL_LIMENG_ST']

svi_county['RPL_THEME3_ST'] = svi_county.groupby('STATE')['SPL_THEME3_ST'].rank(pct=True)
svi_county['EPL_MUNIT_ST'] = svi_county.groupby('STATE')['EP_MUNIT'].rank(pct=True)
svi_county['EPL_MOBILE_ST'] = svi_county.groupby('STATE')['EP_MOBILE'].rank(pct=True)
svi_county['EPL_CROWD_ST'] = svi_county.groupby('STATE')['EP_CROWD'].rank(pct=True)
svi_county['EPL_NOVEH_ST'] = svi_county.groupby('STATE')['EP_NOVEH'].rank(pct=True)
svi_county['EPL_GROUPQ_ST'] = svi_county.groupby('STATE')['EP_GROUPQ'].rank(pct=True)

svi_county['SPL_THEME4_ST'] = svi_county['EPL_MUNIT_ST'] + svi_county['EPL_MOBILE_ST'] + svi_county['EPL_CROWD_ST'] + svi_county['EPL_NOVEH_ST'] + svi_county['EPL_GROUPQ_ST']

svi_county['RPL_THEME4_ST'] = svi_county.groupby('STATE')['SPL_THEME4_ST'].rank(pct=True)

svi_county['SPL_THEMES_ST'] = svi_county['SPL_THEME1_ST'] + svi_county['SPL_THEME2_ST'] + svi_county['SPL_THEME3_ST'] + svi_county['SPL_THEME4_ST']

svi_county['RPL_ThemesStatesCt'] = svi_county.groupby('STATE')['SPL_THEMES_ST'].rank(pct=True)

svi_county.head(1)
svi_county.shape

(3220, 151)

In [634]:
# Create a dictionary for county SVI
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
SVI_county = dict(zip(svi_county.FIPS, svi_county.RPL_ThemesStatesCt))
#len(SVI_county)

## Community Health Workers Coverage Model

### Objective and Prerequisites

In this model, we solve a version of capacitated coverage model: how to allocate community health workers to maximize the coverage in the census tract level. We consider 2-level allocation. First, we allocate community health care workers within counties per state and then census tracts within per county in the state. We implement this model in the Gurobi Python interface and compute optimal solution.

### Motivation
While there has been tremendous emphasis across the country on scaling up contact tracing for COVID-19, exposure notification is only one part of what is needed in communities hardest hit by the pandemic. We need a new politics of care, which addresses the larger health, social and economic issues that many places around the country were struggling with long before COVID-19 and that created health disparities that have lingered for generations. A community health worker approach, which combines basic public health tasks, like contact tracing, health education, social work and legal advocacy, are required to rebuild communities in the wake of COVID-19 and ensure better health for all for the long term.


### Problem Description
![Contact Tracing Allocation](Data/img1.png)

### Model Formulation 
 
#### Sets
$I $ set of contact tracing demand points (census tracts)<br>
$J$  set of contact tracer center locations (counties)<br>
$S$  set of states<br>
$N_i$ set of contact tracer center locations that can serve demand point $i \in I$<br>
$M_j$ set of contact tracer center locations that belongs to state $s \in S$<br>

#### Parameters
$w_{i}$  weight assigned to demand point $i \in I$ <br>
$d_{i}$  total number of contact tracer needed at demand point $i \in I$<br>
$z_{j}$  weight assigned to contact tracer location  $j \in J$<br>
$c_{s}$  total number of contact tracer capacity at each state $s \in S$<br>


The decision variables for the model are as follows: <br>
 $y_{j} = $ the number of contact tracer at center $j \in J$. <br>
 $x_{ij} = $ the number of community health care workers assigned to demand point $i \in I$ by contact tracing center $j \in J$, and 0 otherwise. <br>



The integer programming formulation of our model is as follows.
<br>
$$
\begin{align}
    \max & \sum_{j \in J} \sum_{i \in I} w_{i}x_{ij} + \sum_{j \in J} z_{j}y_{j} && \\
    \text{s.t. } &\sum_{j \in N_{i}}  x_{ij} \leq  d_i &\text{ for }& i \in I  \\
    & \sum_{i \in I: j \in N_{i}} x_{ij} \leq y_{j} \ &\text{ for }& j \in J \\
    & \sum_{j \in M_s}  y_{j} \leq c_s &\text{ for }& s \in S  \\
    & y_{j} \leq d_j &\text{ for }& j \in J  \\
    & x_{ij} \geq 0 \text{ }&\text{  for }& i\in I, j \in J \\
    & y_{j}  \text{ integer }&  \text{ for } &j \in J 
\end{align}
$$

In [635]:
##############################################################
## We first calculate demand for community health care workers
## Demand calculation code is written by Dare Brawley using the 
## George Washington https://www.gwhwi.org/estimator-613404.html 
## calculation
## github: darebrawley
##############################################################

In [636]:
#### replace with desired (or most recent) date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today #or enter a specific date such as '2020-07-06'

#### Parameters for GW model

# base_need = contact tracers per total population 

# We consider several different base: 10,20,30,...80
base_need = 10

# contacts = estimated contacts per case (depends on social distancing measures)
contacts = 10

# case_interviews = number of confirmed case interviews per day  
case_interviews = 6

# cont_notifications = number of contact notifications per day  
cont_notifications = 12

# contact_followup = number of contact follow ups per day  
contact_followup = 32

# follow_freq = follow up frequency (per contact per week)  
followup_freq = 7

In [637]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [638]:
import requests
import io

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))
covid.head(10)


covid[covid['state'] == "Puerto Rico"]
covid.drop(columns = ['deaths'])

,date,county,state,fips,cases
0,2020-01-21,Snohomish,Washington,53061.0,1
1,2020-01-22,Snohomish,Washington,53061.0,1
2,2020-01-23,Snohomish,Washington,53061.0,1
3,2020-01-24,Cook,Illinois,17031.0,1
4,2020-01-24,Snohomish,Washington,53061.0,1
5,2020-01-25,Orange,California,6059.0,1
6,2020-01-25,Cook,Illinois,17031.0,1
7,2020-01-25,Snohomish,Washington,53061.0,1
8,2020-01-26,Maricopa,Arizona,4013.0,1
9,2020-01-26,Los Angeles,California,6037.0,1


In [639]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/boro/boroughs-case-hosp-death.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))
covid_ny.head(10)

# We only have live url for Puerto Rico for now
#live_url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/live/us-counties.csv"

#covid[covid['state'] == "Puerto Rico"]
#covid.sort_values(by="cases").head(5)
#covid_ny.head(5)


,DATE_OF_INTEREST,BK_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BX_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,MN_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,QN_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,SI_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT
0,02/29/2020,0,4,0,0,3,0,1,2,0,0,2,0,0,0,0
1,03/01/2020,0,0,0,0,1,0,0,0,0,0,2,0,0,1,0
2,03/02/2020,0,4,0,0,9,0,0,4,0,0,5,0,0,0,0
3,03/03/2020,0,5,0,1,7,0,0,2,0,1,5,0,0,1,0
4,03/04/2020,1,5,0,0,6,0,2,6,0,2,3,0,0,2,0
5,03/05/2020,3,5,0,0,2,0,0,6,0,0,6,0,0,1,0
6,03/06/2020,1,3,0,2,5,0,3,3,0,1,5,0,1,2,0
7,03/07/2020,2,6,0,0,2,0,1,4,0,3,3,0,1,0,0
8,03/08/2020,5,9,0,4,2,0,6,1,0,5,5,0,1,0,0
9,03/09/2020,16,15,0,4,11,0,25,12,0,10,8,0,3,3,0


In [640]:
Kings = covid_ny[['DATE_OF_INTEREST', 'BK_CASE_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = '36047'
Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,date,cases,county,state,fips
0,02/29/2020,0,Kings,New York,36047
1,03/01/2020,0,Kings,New York,36047
2,03/02/2020,0,Kings,New York,36047
3,03/03/2020,0,Kings,New York,36047
4,03/04/2020,1,Kings,New York,36047


In [641]:
Bronx = covid_ny[['DATE_OF_INTEREST', 'BX_CASE_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = '36005'
Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

,date,cases,state,county,fips
0,02/29/2020,0,New York,Bronx,36005
1,03/01/2020,0,New York,Bronx,36005
2,03/02/2020,0,New York,Bronx,36005
3,03/03/2020,1,New York,Bronx,36005
4,03/04/2020,0,New York,Bronx,36005


In [642]:
Manhattan = covid_ny[['DATE_OF_INTEREST', 'MN_CASE_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = '36061'
Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

,date,cases,state,county,fips
0,02/29/2020,1,New York,Manhattan,36061
1,03/01/2020,0,New York,Manhattan,36061
2,03/02/2020,0,New York,Manhattan,36061
3,03/03/2020,0,New York,Manhattan,36061
4,03/04/2020,2,New York,Manhattan,36061


In [643]:
Queens = covid_ny[['DATE_OF_INTEREST', 'QN_CASE_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = '36081'
Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

,date,cases,state,county,fips
0,02/29/2020,0,New York,Queens,36081
1,03/01/2020,0,New York,Queens,36081
2,03/02/2020,0,New York,Queens,36081
3,03/03/2020,1,New York,Queens,36081
4,03/04/2020,2,New York,Queens,36081


In [644]:
Richmond = covid_ny[['DATE_OF_INTEREST', 'SI_CASE_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = '36085'
Richmond.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

,date,cases,state,county,fips
0,02/29/2020,0,New York,Richmond,36085
1,03/01/2020,0,New York,Richmond,36085
2,03/02/2020,0,New York,Richmond,36085
3,03/03/2020,0,New York,Richmond,36085
4,03/04/2020,0,New York,Richmond,36085


In [645]:
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)
covid.head(10)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0.0
1,2020-01-22,Snohomish,Washington,53061,1,0.0
2,2020-01-23,Snohomish,Washington,53061,1,0.0
3,2020-01-24,Cook,Illinois,17031,1,0.0
4,2020-01-24,Snohomish,Washington,53061,1,0.0
5,2020-01-25,Orange,California,6059,1,0.0
6,2020-01-25,Cook,Illinois,17031,1,0.0
7,2020-01-25,Snohomish,Washington,53061,1,0.0
8,2020-01-26,Maricopa,Arizona,4013,1,0.0
9,2020-01-26,Los Angeles,California,6037,1,0.0


In [646]:

covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)

In [647]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [648]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)
#covid_last15.head(10)
#covid_last15[covid_last15['state'] == "New York"]
#covid_last15[covid_last15['state'] == "Puerto Rico"]

In [649]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()
covid_last14.sort_values(by="cases").head(10)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
136,07/14/2020,Richmond,New York,36085,0,NaN,2020-07-14,-1 days,0.0
329652,2020-07-13,Chase,Nebraska,31029,1,0.0,2020-07-13,-2 days,0.0
332169,2020-07-14,Washington,Kansas,20201,1,0.0,2020-07-14,-1 days,0.0
311721,2020-07-07,Throckmorton,Texas,48447,1,0.0,2020-07-07,-8 days,0.0
296378,2020-07-03,Kusilvak Census Area,Alaska,2158,1,0.0,2020-07-03,-12 days,NaN
296370,2020-07-03,Denali Borough,Alaska,2068,1,0.0,2020-07-03,-12 days,0.0
307762,2020-07-06,Nelson,North Dakota,38063,1,0.0,2020-07-06,-9 days,0.0
311672,2020-07-07,Motley,Texas,48345,1,0.0,2020-07-07,-8 days,0.0
311666,2020-07-07,Mills,Texas,48333,1,0.0,2020-07-07,-8 days,0.0
301051,2020-07-04,Beaverhead,Montana,30001,1,0.0,2020-07-04,-11 days,0.0


In [650]:
## calculate follow up encounter demand
covid_last14['fe_demand'] = covid_last14['new_cases']*(14 + ((covid_last14['dt'] - data_date_dt).dt.days))/7*followup_freq*contacts

In [651]:
## group by to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases','fe_demand'].sum()

In [652]:
#svi_county.head(5)
#covid_last14_stats.head(5)
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'left', right_on = 'FIPS', left_on = 'fips' )


In [653]:
covid_last14_stats.head(5)

,fips,new_cases,fe_demand,E_TOTPOP,FIPS,STATE
0,1001,193.0,14480.0,55200.0,1001.0,ALABAMA
1,1003,711.0,51800.0,208107.0,1003.0,ALABAMA
2,1005,102.0,8050.0,25782.0,1005.0,ALABAMA
3,1007,62.0,4340.0,22527.0,1007.0,ALABAMA
4,1009,148.0,12890.0,57645.0,1009.0,ALABAMA


In [654]:
#x = covid_last15.groupby('fips')['cases'].sum()
#print (x)


In [655]:
## calculations for each type of demand
covid_last14_stats['base_ct'] = (covid_last14_stats['E_TOTPOP']/100000)*base_need
covid_last14_stats['case_int_need'] = covid_last14_stats['new_cases']/case_interviews/5
covid_last14_stats['contact_notify_need'] = (covid_last14_stats['new_cases']*contacts/cont_notifications)/5
covid_last14_stats['contact_follow_need'] = covid_last14_stats['fe_demand']/(contact_followup*5*2)
covid_last14_stats['covid_need'] = covid_last14_stats['case_int_need']+covid_last14_stats['contact_notify_need']+covid_last14_stats['contact_follow_need']
covid_last14_stats['base_ct'] = covid_last14_stats['base_ct'].round()
covid_last14_stats['covid_need'] = covid_last14_stats['covid_need'].round()
covid_last14_stats['total_need'] = np.where(covid_last14_stats['covid_need'] < 1e-6, 0, covid_last14_stats['covid_need'])
#covid_last14_stats['total_need'] = max(covid_last14_stats['covid_need'],1)


In [656]:
#covid[covid['state'] == "New York"]

In [657]:
#covid_last14_stats[covid_last14_stats['STATE'] == "NEW YORK"]

In [658]:
#svi_county[svi_county['STATE'] == "NEW YORK"]

In [659]:
covid_last14_stats.sort_values(by="new_cases").head(5)


,fips,new_cases,fe_demand,E_TOTPOP,FIPS,STATE,base_ct,case_int_need,contact_notify_need,contact_follow_need,covid_need,total_need
2325,46021,0.0,0.0,1435.0,46021.0,SOUTH DAKOTA,0.0,0.0,0.0,0.0,0.0,0.0
2979,54089,0.0,0.0,13018.0,54089.0,WEST VIRGINIA,1.0,0.0,0.0,0.0,0.0,0.0
555,16025,0.0,0.0,886.0,16025.0,IDAHO,0.0,0.0,0.0,0.0,0.0,0.0
2107,40043,0.0,0.0,4918.0,40043.0,OKLAHOMA,0.0,0.0,0.0,0.0,0.0,0.0
1722,32029,0.0,0.0,3941.0,32029.0,NEVADA,0.0,0.0,0.0,0.0,0.0,0.0


In [660]:
#x = covid_last14_stats.groupby(['STATE'])['total_need'].sum()
#print (x)

In [661]:
## saving simplified form
covid_demand = covid_last14_stats[['fips','total_need']]
covid_demand.sort_values(by="total_need").tail(100)
covid_demand.sort_values(by="fips").tail(10)
#print(len(covid_last14_stats[covid_last14_stats.total_need > 0]))
#print (len(covid_last14_stats))

# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.new_cases))

# Create a dictionary for the county and CHW demand
CHW_demand_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_need))

covid_cases_county = {}
CHW_demand_county = {}

for j in SVI_county:
    #print ('svi',j)
    if j in covid_cases_county_ny_times:
        #print('co', j)
        #print (covid_cases_county_ny_times[j])
        #print (CHW_demand_county_ny_times[j])
        covid_cases_county[j] = covid_cases_county_ny_times[j]
        CHW_demand_county[j] = CHW_demand_county_ny_times [j]
    else:
        covid_cases_county[j] = -999
        CHW_demand_county[j] = -999
        



In [662]:
############################################################################
############ END of demand calculation #####################################

In [663]:
#df_1 = pd.read_csv('County_based_demand.csv')

df = covid_last14_stats

In [664]:
#county_svi = pd.read_csv('Data/SVI_county_based_per_State.csv', encoding='latin-1')

In [665]:
#TextFileReader = pd.read_csv('Data/svi_2018_tracts_state_ranked.csv',chunksize=5000)

#df_svi_x = []

#for df in TextFileReader:
#    df_svi_x.append(df)
    
#df_svi = pd.concat(df_svi_x, sort=False)


In [666]:
#df_2 = pd.merge(left = df_1, right = county_svi, how = 'right', right_on = 'FIPS', left_on = 'FIPS' )

In [667]:
#df = pd.merge(left = df_2, right = df_svi, how = 'right', right_on = 'STCNTY', left_on = 'FIPS' )

#df['RPL_ThemesStates_y'] = df['RPL_ThemesStates_y'].fillna(0)

df.head(10)
df['fips'] = df['fips'].astype(int)
svi_census['STCNTY'] = svi_census['STCNTY'].astype(int)


In [668]:
#print (svi_county['FIPS'])
df_2 = pd.merge(left = df, right = svi_census, how = 'right', right_on = 'STCNTY', left_on = 'fips' )

df_2.rename(columns={'FIPS_x': 'FIPS_county', 'STATE_x':'STATE', 'FIPS_y': 'FIPS_census', 'E_TOTPOP_x' : 'E_TOTPOP_county', 'E_TOTPOP_y' : 'E_TOTPOP_census'}, inplace=True)
df_2.head(5)
#df_2[df_2['STATE'] == "PUERTO RICO"]

,fips,new_cases,fe_demand,E_TOTPOP_county,FIPS_county,STATE,base_ct,case_int_need,contact_notify_need,contact_follow_need,covid_need,total_need,ST,STATE_y,ST_ABBR,STCNTY,COUNTY,FIPS_census,LOCATION,AREA_SQMI,E_TOTPOP_census,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry,EPL_POV_ST,EPL_UNEMP_ST,EPL_PCI_ST,EPL_NOHSDP_ST,SPL_THEME1_ST,RPL_THEME1_ST,EPL_AGE65_ST,EPL_AGE17_ST,EPL_DISABL_ST,EPL_SNGPNT_ST,SPL_THEME2_ST,RPL_THEME2_ST,EPL_MINRTY_ST,EPL_LIMENG_ST,SPL_THEME3_ST,RPL_THEME3_ST,EPL_MUNIT_ST,EPL_MOBILE_ST,EPL_CROWD_ST,EPL_NOVEH_ST,EPL_GROUPQ_ST,SPL_THEME4_ST,RPL_THEME4_ST,SPL_THEMES_ST,RPL_ThemesStates
0,1001.0,193.0,14480.0,55200.0,1001.0,ALABAMA,6.0,6.433333,32.166667,45.25,84.0,84.0,01,ALABAMA,AL,1001,Autauga,1001020100,"Census Tract 201, Autauga County, Alabama",3.790677,1923,253,779,69,765,73,218,109,35,23,31580,5144,123,46,245,52,407,100,339,87,66,44.1,382,332.0,14,49.2,0,17.0,135,56,3,13.0,6,9,0,12,11.3,5.8,3.6,2.4,31580.0,5144.0,9.4,3.6,12.7,2.9,21.2,4.4,17.6,4.5,8.6,5.7,19.9,17.1,0.8,2.7,0.0,2.2,17.3,7.2,0.4,1.7,0.8,1.2,0.0,0.6,0.4701,0.2724,0.4033,0.4649,1.6107,0.3913,0.3451,0.4188,0.7897,0.5502,2.1038,0.5597,0.3717,0.3889,0.7606,0.3735,0.0000,0.8723,0.1999,0.0696,0.0000,1.1418,0.0993,5.6169,0.2533,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,178,79,9.3,4.1,1018,0.000948,0.150082,"POLYGON ((-86.50916 32.47344, -86.50620 32.475...",0.266949,0.209322,0.811864,0.263983,1.552119,0.222034,0.223729,0.400000,0.547881,0.520763,1.692373,0.290678,0.346186,0.689831,1.036017,0.512712,0.171186,0.636864,0.282627,0.068220,0.257627,1.416525,0.063559,5.697034,0.139831
1,1001.0,193.0,14480.0,55200.0,1001.0,ALABAMA,6.0,6.433333,32.166667,45.25,84.0,84.0,01,ALABAMA,AL,1001,Autauga,1001020300,"Census Tract 203, Autauga County, Alabama",2.065365,3476,433,1397,92,1296,110,523,248,55,53,22527,2837,307,95,580,87,841,254,735,180,108,72.4,1170,543.2,113,127.8,18,28.6,19,24,28,33.2,99,54,0,12,15.0,6.5,3.3,3.2,22527.0,2837.0,12.8,4.0,16.7,3.3,24.2,6.7,21.2,4.2,8.3,5.5,33.7,15.1,3.3,3.8,1.3,2.0,1.4,1.8,2.2,2.6,7.6,4.1,0.0,0.3,0.6033,0.2320,0.7218,0.6086,2.1657,0.5659,0.5877,0.6384,0.9040,0.5294,2.6595,0.8729,0.5424,0.6799,1.2223,0.6429,0.3297,0.5669,0.5416,0.6300,0.0000,2.0682,0.3882,8.1157,0.6400,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,119,81,3.4,2.2,1836,0.000516,0.100175,"POLYGON ((-86.47087 32.47573, -86.46964 32.478...",0.399153,0.175000,0.467797,0.377542,1.419492,0.142373,0.491949,0.685593,0.751695,0.494915,2.424153,0.764407,0.549576,0.910169,1.459746,0.833898,0.523729,0.286441,0.689831,0.663983,0.257627,2.421610,0.444915,7.725000,0.539831
2,1001.0,193.0,14480.0,55200.0,1001.0,ALABAMA,6.0,6.433333,32.166667,45.25,84.0,84.0,01,ALABAMA,AL,1001,Autauga,1001020400,"Census Tract 204, Autauga County, Alabama",2.464982,3831,337,1867,106,1639,144,109,70,49,40,30527,3194,165,60,901,119,785,135,742,146,104,72.3,470,449.2,51,74.2,72,67.4,0,12,13,16.4,21,24,0,12,2.8,1.8,2.9,2.3,30527.0,3194.0,6.0,2.2,23.5,3.4,20.5,3.0,19.6,3.7,6.3,4.4,12.3,11.7,1.4,2.1,3.9,3.

In [669]:
# Create a dictionary for the county and population
population_county = dict(zip(svi_county.FIPS, svi_county.E_TOTPOP))

# Create a dictionary for the census tract and population
population_census = dict(zip(svi_census.FIPS, svi_census.E_TOTPOP))

# Create a dictionary for the name and FIPS of the county
FIPS_census_county_dict = dict(zip(svi_census.STCNTY, svi_census.COUNTY))

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_census.STCNTY, svi_census.STATE))

# Create a dictionary for census tract FIPS and county FIPS
census_county = dict(zip(svi_census.FIPS, svi_census.STCNTY))


# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = [k for k in SVI_county ]


# Create a dictionary for the demand per county
# We use George Washington https://www.gwhwi.org/estimator-613404.html
# to calculate the demand
county_demand_14 = dict(zip(df.fips, df.total_need))

county_demand = {}

for j in location:
    if j in county_demand_14 and county_demand_14[j] > 1e-10:
        county_demand[j] = county_demand_14[j]
    else:
        county_demand[j] = 0
       

# Create demand point list from the census tract FIPS 
df_2['FIPS_census'] = df_2['FIPS_census'].astype(int)

demand_point = df_2.FIPS_census.tolist()


In [670]:
# Each census only can be covered by its county so we simple set it to coverage set to the county of that census
coverage_loc_set = census_county


# since we have the demand data in the county level, to create the demand in the census tract level, we simple use the
# ratio of the census population to the county population
demand = {}
for d in demand_point:
    
    if  population_county[census_county[d]] >= 1e-6:
        demand[d] = (county_demand[census_county[d]]*population_census[d])/population_county[census_county[d]]       
    else:
        demand[d] = 0     
    


In [671]:
#demand_point, demand = gp.multidict({demand_p:demand} for demand in demand_county for demand_p in demand_point_x)
new_df = df_2.drop_duplicates(subset=['fips'])
#new_df.dropna(subset=['STATE'])
State = list(set(svi_county.STATE.tolist()))
State = [s for s in State if s != math.nan]
capacity = new_df.groupby(['STATE'])['base_ct'].sum().to_dict()


In [672]:
demand_per_State = new_df.groupby(['STATE'])['total_need'].sum().to_dict()
population_per_state =  new_df.groupby(['STATE'])['E_TOTPOP_county'].sum().to_dict()

##########################################
# Change CT needed per 100,000 population
# For we can set it to low = 30, medium = 50, 
# high = 70 CT or any number in the set {0, 10, ..., 100) CT per 100,000
capacity_multiplier = 30

######################################
# Or specify certain number of CT per state
minimum_contact_tracer_per_state = {s: 1 for s in State}
# print (minimum_contact_tracer_per_state)

capacity_x = {}
#print (State)
for s in capacity:
    capacity_x[s] = math.ceil(max((capacity_multiplier*population_per_state[s])/100000, minimum_contact_tracer_per_state[s]))
    

In [673]:
#Parameters
#location = list(dict.fromkeys(location))
pro = [(i,coverage_loc_set[i]) for i in demand_point]
#print (location)
#print (pro)
pro_c_s = [(i,county_of_states[i]) for i in location]
#print (pro_c_s)

In [674]:

cartesian_prod = gp.tuplelist(pro)
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [675]:
#MIP model formulation
m = gp.Model("Contact_Tracing_Coverage")

In [676]:
#Add variable for each contact tracer center
y = m.addVars(location, vtype = GRB.INTEGER, name = "y")

#Add variable for each demand point (census tract)
x = m.addVars(cartesian_prod, vtype = GRB.CONTINUOUS, name = "x")



In [677]:
# Coverage constraint
m.addConstrs((gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select(i,'*')) <= demand[i] for i in demand_point), name = 'census_tract_demand')
m.update()

In [678]:
#CT allocation for all demand point (census tracts) in a county cannot exceed the allocation to the county
m.addConstrs((gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select('*',j)) <=   y[j] for j in location), name = 'census_tract_and_county_allocation')
m.update()

In [679]:
# Each state has a predefined capacity. The allocation of CT in all counties for each state cannot exceed the state's capacity
capacity_const = {}
for s in capacity:
    capacity_const[s] = m.addConstr((gp.quicksum(y[j] for j in location if (j,s) in cartesian_pro_county_state) <= (capacity[s])), name = 'State_capacity')
m.update()
    
#for s in capacity:
#    print (capacity_const[s])
 #   print (capacity_const[s].rhs)


In [680]:
m.addConstrs((y[j] <=   county_demand[j] for j in location), name = 'County_demand')
m.update()

In [681]:
counties_in_state = {s:[] for s in State}
for j in location:
    counties_in_state[county_of_states[j]].append(j)

census_in_county = {j: [] for j in location}

for i in demand_point:
    census_in_county[census_county[i]].append(i)
#print (census_in_county)


In [682]:
min_SVI_pop_census = {j: min(SVI_census[i]*population_census[i] for i in census_in_county[j]) for j in location }
max_SVI_pop_census = {j: max(SVI_census[i]*population_census[i] for i in census_in_county[j]) for j in location }

#print (min_SVI_pop_census)

#print (min_SVI_pop_census)

In [683]:

SVI_county = {k: SVI_county[k] for k in SVI_county if not math.isnan(SVI_county[k])}

hotspot = {} #demand divided by pop if no pop then it is 0
total = 0
for j in location:
    #print (county_demand[j],j)
    
    if population_county[j] <1e-6:
        hotspot[j] = 0
    else:
        hotspot[j] = county_demand[j]/population_county[j]
        total += hotspot[j]
        #print(j, county_demand[j], demand_per_county[j], population_county[j])


######################################################################
## Normalize the weights within each state we use for different policies
## First we need max and min for each different policy
min_demand = {s: min(covid_cases_county[j] for j in counties_in_state[s]) for s in counties_in_state }
max_demand = {s: max(covid_cases_county[j] for j in counties_in_state[s]) for s in counties_in_state }

min_SVI_hotspot = {s: min(SVI_county[j]*hotspot[j] for j in counties_in_state[s]) for s in counties_in_state }
max_SVI_hotspot = {s: max(SVI_county[j]*hotspot[j] for j in counties_in_state[s]) for s in counties_in_state }

min_SVI_pop = {s: min(population_county[j]*SVI_county[j] for j in counties_in_state[s]) for s in counties_in_state }
max_SVI_pop = {s: max(population_county[j]*SVI_county[j] for j in counties_in_state[s]) for s in counties_in_state }

min_hotspot = {s: min(hotspot[j] for j in counties_in_state[s]) for s in counties_in_state }
max_hotspot = {s: max(hotspot[j] for j in counties_in_state[s]) for s in counties_in_state }

min_SVI_high_demand = {s: min(SVI_county[j]*covid_cases_county[j] for j in counties_in_state[s]) for s in counties_in_state }
max_SVI_high_demand = {s: max(SVI_county[j]*covid_cases_county[j] for j in counties_in_state[s]) for s in counties_in_state }

high_demand_normalized = {}
SVI_hotspot_normalized = {}
SVI_pop_normalized = {}
hotspot_normalized = {}
SVI_high_demand_normalized = {}

census_SVI_pop_normalized = {}

high_demand = {}
SVI_hotspot = {}
SVI_pop = {}
SVI_high_demand = {}

#print (location)

for j in location:
    
    high_demand[j] = covid_cases_county[j]
    
    SVI_hotspot[j] = SVI_county[j]*hotspot[j]
    
    SVI_pop[j] = population_county[j]*SVI_county[j]
     
    SVI_high_demand[j] = SVI_county[j]*covid_cases_county[j]
    
    #print (j, SVI_pop[j], SVI_hotspot[j])



high_demand = {k: high_demand[k] for k in high_demand if not math.isnan(k)}


for j in location:
    
    if (max_demand[county_of_states[j]] - min_demand[county_of_states[j]]) != 0:
    
        high_demand_normalized[j] = (covid_cases_county[j] - min_demand[county_of_states[j]])/(max_demand[county_of_states[j]] - min_demand[county_of_states[j]])
    
    else: 
        high_demand_normalized[j] = 1
        
    if (max_SVI_hotspot[county_of_states[j]] - min_SVI_hotspot[county_of_states[j]])!= 0:
        
        SVI_hotspot_normalized[j] = (SVI_county[j]*hotspot[j] - min_SVI_hotspot[county_of_states[j]])/(max_SVI_hotspot[county_of_states[j]] - min_SVI_hotspot[county_of_states[j]])
    
    else:
        SVI_hotspot_normalized[j] = 1
    
    if (max_SVI_pop[county_of_states[j]] - min_SVI_pop[county_of_states[j]]) != 0:
        
        SVI_pop_normalized[j] = (population_county[j]*SVI_county[j] - min_SVI_pop[county_of_states[j]])/(max_SVI_pop[county_of_states[j]] - min_SVI_pop[county_of_states[j]])
    
    else:
        SVI_pop_normalized[j] = 1
    
    if (max_hotspot[county_of_states[j]] - min_hotspot[county_of_states[j]]) != 0:
        
        hotspot_normalized[j] = (hotspot[j] - min_hotspot[county_of_states[j]])/(max_hotspot[county_of_states[j]] - min_hotspot[county_of_states[j]])
    
    else: 
        hotspot_normalized[j] = 1
        
    if (max_SVI_high_demand[county_of_states[j]] - min_SVI_high_demand[county_of_states[j]]) != 0:
        
        SVI_high_demand_normalized[j] = (SVI_county[j]*covid_cases_county[j] - min_SVI_high_demand[county_of_states[j]])/(max_SVI_high_demand[county_of_states[j]] - min_SVI_high_demand[county_of_states[j]])
    
    else:
        SVI_high_demand_normalized[j] = 1
        
min_SVI_pop_census = {j: min(SVI_census[i]*population_census[i] for i in census_in_county[j]) for j in location }
max_SVI_pop_census = {j: max(SVI_census[i]*population_census[i] for i in census_in_county[j]) for j in location }

for i in demand_point:
    
    
    if (max_SVI_pop_census[census_county[i]] - min_SVI_pop_census[census_county[i]]) != 0:
    
        census_SVI_pop_normalized[i] = (SVI_census[i]*population_census[i] - min_SVI_pop_census[census_county[i]])/(max_SVI_pop_census[census_county[i]] - min_SVI_pop_census[census_county[i]])
    else:
        
        census_SVI_pop_normalized[i] = 1
        

In [684]:
# We change the weight in the objective for different policies
# (1) demand_per_county[j] prioritizes the counties with high demand first within per state to allocate contact tracers

# (2) (SVI*hotspot = SVI_county[j]*hot_stop_weight[j]*100000) this one we prioritize SVI*hot spot counties 

# (3) (SVI*pop for county = SVI_county[j]*population_county[j]) prioritize SVI and high pop areas 

# (4) hot_spot_weight[j]*100000 this one prioritize hot spot counties first


base_case_capacity = {i: i for i in range(1,9)}

print (base_case_capacity)

Strategies = {'high_demand': high_demand, 'SVI_high_demand': SVI_high_demand, 'SVI_hotspot': SVI_hotspot, 'SVI_pop': SVI_pop, 'hotspot': hotspot}

m.update()

percentage_census = {}
percentage_county = {}

# We consider different level for base case capacity 10,20,30,...80 per 100,000



for s in Strategies:
    
    for c in base_case_capacity:
        percentage_census[s,c] = {}
        percentage_county[s,c] = {}
        
        
        for p in capacity:

        
            capacity_const[p].RHS = capacity[p]*base_case_capacity[c]
    
        m.setObjective( gp.quicksum((1e-6)*SVI_census[i]*population_census[i]*x[i,j] for (i,j) in cartesian_prod  if not math.isnan(SVI_census[i]))
 + gp.quicksum(Strategies[s][j]*y[j] for j in location), GRB.MAXIMIZE)


        m.update()
        
        print ('\n************ Strategy',s,'******* Base case capacity*******', c)
        m.update()
        m.optimize()
        
        for i in demand_point:
            if demand[i] > 0:
                #print (i)
                percentage_census[s,c][i] = (100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i]) 
                
            else:
                percentage_census[s,c][i] = -1
        
        
        
        for j in location:
            if county_demand[j] > 1e-6:
                percentage_county[s,c][j] = (y[j].x/county_demand[j])*100
            else:
                percentage_county[s,c][j] = -1
                
        #print (percentage[s,c])
        m.reset()


{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8}

************ Strategy high_demand ******* Base case capacity******* 1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 80237 rows, 76965 columns and 157150 nonzeros
Model fingerprint: 0x6e60c277
Variable types: 73745 continuous, 3220 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-09, 3e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-05, 2e+04]
Found heuristic solution: objective 2.793355e+07
Presolve removed 80183 rows and 70095 columns
Presolve time: 0.56s
Presolved: 54 rows, 6870 columns, 6923 nonzeros
Found heuristic solution: objective 1.960799e+08
Variable types: 6817 continuous, 53 integer (0 binary)

Root relaxation: objective 3.171504e+08, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0        

Variable types: 73745 continuous, 3220 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-09, 3e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-05, 3e+04]
Found heuristic solution: objective 8.637036e+08
Presolve removed 80182 rows and 68546 columns
Presolve time: 0.20s
Presolved: 55 rows, 8419 columns, 8473 nonzeros
Found heuristic solution: objective 1.604154e+09
Variable types: 8365 continuous, 54 integer (0 binary)

Root relaxation: objective 1.656271e+09, 51 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1.656271e+09 1.6563e+09 -0.00%     -    0s

Explored 0 nodes (51 simplex iterations) in 0.26 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 1.65627e+09 1.60415e+09 8.63704e+08 

Optimal solution found (tolerance 1.00e-04)
Best object

Presolve removed 80182 rows and 68546 columns
Presolve time: 0.22s
Presolved: 55 rows, 8419 columns, 8473 nonzeros
Found heuristic solution: objective 8.971094e+08
Variable types: 8365 continuous, 54 integer (0 binary)

Root relaxation: objective 1.091934e+09, 53 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1.091934e+09 1.0919e+09 -0.00%     -    0s

Explored 0 nodes (53 simplex iterations) in 0.28 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 1.09193e+09 8.97109e+08 3.85528e+08 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.091933792677e+09, best bound 1.091933792677e+09, gap 0.0000%
Discarded solution information

************ Strategy SVI_high_demand ******* Base case capacity******* 6
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 80237 rows, 76965

Presolved: 54 rows, 7788 columns, 7841 nonzeros
Found heuristic solution: objective 485.3464968
Variable types: 7735 continuous, 53 integer (0 binary)

Root relaxation: objective 5.319017e+02, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  531.90173    0   34  485.34650  531.90173  9.59%     -    0s
H    0     0                     531.8513298  531.90173  0.01%     -    0s

Cleanup yields a better solution

Explored 1 nodes (11 simplex iterations) in 0.61 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 531.851 485.346 265.309 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.318612714792e+02, best bound 5.319017251912e+02, gap 0.0076%
Discarded solution information

************ Strategy SVI_hotspot ******* Base case capacity******* 4
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a mod

Model fingerprint: 0x2f710eda
Variable types: 73745 continuous, 3220 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-09, 9e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-05, 2e+04]
Found heuristic solution: objective 5.963177e+09
Presolve removed 80183 rows and 70095 columns
Presolve time: 0.38s
Presolved: 54 rows, 6870 columns, 6923 nonzeros
Found heuristic solution: objective 3.949119e+10
Variable types: 6817 continuous, 53 integer (0 binary)

Root relaxation: objective 7.151214e+10, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7.151214e+10 7.1512e+10 -0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.44 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 7.15121e+10 3.94912e+10 5.96318e+09 

Optimal solution found (tol

Presolve removed 80182 rows and 68546 columns
Presolve time: 0.19s
Presolved: 55 rows, 8419 columns, 8473 nonzeros
Found heuristic solution: objective 2.836463e+11
Variable types: 8365 continuous, 54 integer (0 binary)

Root relaxation: objective 2.902367e+11, 51 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2.902367e+11 2.9024e+11 -0.00%     -    0s

Explored 0 nodes (51 simplex iterations) in 0.26 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 2.90237e+11 2.83646e+11 1.99313e+11 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.902367218849e+11, best bound 2.902367218849e+11, gap 0.0000%
Discarded solution information

************ Strategy SVI_pop ******* Base case capacity******* 8
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 80237 rows, 76965 columns

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-09, 3e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-05, 2e+04]
Found heuristic solution: objective 515.5466430
Presolve removed 80182 rows and 68546 columns
Presolve time: 0.39s
Presolved: 55 rows, 8419 columns, 8473 nonzeros
Found heuristic solution: objective 739.0657078
Variable types: 8365 continuous, 54 integer (0 binary)

Root relaxation: objective 7.934175e+02, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  793.41749    0   43  739.06571  793.41749  7.35%     -    0s
H    0     0                     793.3229528  793.41749  0.01%     -    0s
H    0     0                     793.4072505  793.41749  0.00%     -    0s

Explored 1 nodes (58 simplex iterations) in 0.46 seconds
Thread count was 8 (of 8 available processors)

Solution count 4: 793

In [685]:
#Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [686]:
# Write csv file for census tracts


fieldnames = []  
write_census_row = {}
fieldnames.append('Census_Tract_FIPS')
fieldnames.append('priority')
fieldnames.append('SVI_census')

SVI_values = {i:SVI_census[i] for i in demand_point}
s_count = 1
for s in Strategies:   
    for c in base_case_capacity:
        d = c*10
        d = str(d)
        fieldnames.append('percentage_scenario_' + s + '_base_case_capacity_' + d)

        write_census_row[s_count] = (percentage_census[s,c].values())
        s_count += 1

writefile = 'Output/Census_tract_level_coverage_for_all_policies_and_different_base_case_capacity.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)  
    writer.writerow(fieldnames)

    for row in zip(demand_point, census_SVI_pop_normalized.values(), SVI_values.values(), write_census_row[1], write_census_row[2], write_census_row[3],write_census_row[4], write_census_row[5], write_census_row[6], write_census_row[7]
                   , write_census_row[8],write_census_row[9],write_census_row[10],write_census_row[11],write_census_row[12]
                  , write_census_row[13], write_census_row[14], write_census_row[15],write_census_row[16], write_census_row[17], write_census_row[18], write_census_row[19]
                   , write_census_row[20],write_census_row[21],write_census_row[22],write_census_row[23],write_census_row[24]
                  , write_census_row[25], write_census_row[26], write_census_row[27],write_census_row[28], write_census_row[29], write_census_row[30], write_census_row[31]
                   , write_census_row[32], write_census_row[33], write_census_row[34], write_census_row[35],write_census_row[36], write_census_row[37], write_census_row[38], write_census_row[39]
                   , write_census_row[40]):                    
        writer.writerow(row)
            
            


In [687]:
# Write file for county 

fieldnames = []  
write_county_row = {}
fieldnames.append('County_FIPS')
fieldnames.append('SVI_county')
fieldnames.append("Covid_cases")
fieldnames.append("CHW_need")
fieldnames.append('priority_high_demand')
fieldnames.append('priority_SVI_high_demand')
fieldnames.append('priority_SVI_hotspot')
fieldnames.append('priority_SVI_pop')
fieldnames.append('priority_hotspot')

SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    for c in base_case_capacity:
        d = c*10
        d = str(d)
        fieldnames.append('percentage_scenario_' + s + '_base_case_capacity_' + d)

        write_county_row[s_count] = (percentage_county[s,c].values())
        s_count += 1

writefile = 'Output/County_level_coverage_for_all_policies_and_different_base_case_capacity.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location, SVI_values.values(), covid_cases_county.values(), CHW_demand_county.values(), high_demand_normalized.values(), SVI_high_demand_normalized.values(),SVI_hotspot_normalized.values(), SVI_pop_normalized.values(), hotspot_normalized.values(),  write_county_row[1], write_county_row[2], write_county_row[3],write_county_row[4], write_county_row[5], write_county_row[6], write_county_row[7]
                   , write_county_row[8],write_county_row[9],write_county_row[10],write_county_row[11],write_county_row[12]
                  ,  write_county_row[13], write_county_row[14], write_county_row[15],write_county_row[16], write_county_row[17], write_county_row[18], write_county_row[19]
                   , write_county_row[20],write_county_row[21],write_county_row[22],write_county_row[23],write_county_row[24]
                  ,write_county_row[25], write_county_row[26], write_county_row[27],write_county_row[28], write_county_row[29], write_county_row[30], write_county_row[31]
                   , write_county_row[32], write_county_row[33], write_county_row[34], write_county_row[35],write_county_row[36], write_county_row[37], write_county_row[38], write_county_row[39]
                   , write_county_row[40]):                    
        writer.writerow(row)